In [ ]:
!pip install ipywidgets==7.6.0

In [ ]:
!pip install -r ../src/requirements.txt

In [ ]:
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.2/892.2 KB 12.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 KB 3.0 MB/s eta 0:00:0000:01
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached schema-0.7.5-py2.py3-none-any.whl (17 kB)
  Using cached PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (736 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 1.7 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 48.5 MB/s eta 0:00:0000:0100:01
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Created wheel for sagemaker: filename=sagemaker-2.188.0-py2.py3-none-any.whl size=1193929 sha256=59f92e13f89d7fee48158d0efa142e0510927beb9f731d9fd1e3bd9f6464c47a
  Stored in directory: /root/.cache/pip/wheels/69/4d/04/6b63459ad89e771129bb0e49d10e64d19358387db5a7d9bbcb
Success

In [ ]:
from datasets import load_from_disk

In [ ]:
data = load_from_disk("./dataset")
data

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 1063
    })
})

In [ ]:
valid_data = data["validation"]
test_data = data["test"]

In [ ]:
valid_data[0]

{'idx': 0,
 'label': 1,
 'sentence': 'The sailors rode the breeze clear of the rocks.'}

In [ ]:
import sagemaker
from sagemaker.predictor import Predictor

sess = sagemaker.Session()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
from src.net import Net

In [ ]:
model = Net()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.state_dict().keys()

odict_keys(['model.distilbert.embeddings.word_embeddings.weight', 'model.distilbert.embeddings.position_embeddings.weight', 'model.distilbert.embeddings.LayerNorm.weight', 'model.distilbert.embeddings.LayerNorm.bias', 'model.distilbert.transformer.layer.0.attention.q_lin.weight', 'model.distilbert.transformer.layer.0.attention.q_lin.bias', 'model.distilbert.transformer.layer.0.attention.k_lin.weight', 'model.distilbert.transformer.layer.0.attention.k_lin.bias', 'model.distilbert.transformer.layer.0.attention.v_lin.weight', 'model.distilbert.transformer.layer.0.attention.v_lin.bias', 'model.distilbert.transformer.layer.0.attention.out_lin.weight', 'model.distilbert.transformer.layer.0.attention.out_lin.bias', 'model.distilbert.transformer.layer.0.sa_layer_norm.weight', 'model.distilbert.transformer.layer.0.sa_layer_norm.bias', 'model.distilbert.transformer.layer.0.ffn.lin1.weight', 'model.distilbert.transformer.layer.0.ffn.lin1.bias', 'model.distilbert.transformer.layer.0.ffn.lin2.weigh

In [ ]:
!aws s3 cp "s3://sagemaker-us-east-1-046017406246/project-3-cola/cola-distilbert-model-2023-09-30-15-01-17-574/output/model.tar.gz" .

download: s3://sagemaker-us-east-1-046017406246/project-3-cola/cola-distilbert-model-2023-09-30-15-01-17-574/output/model.tar.gz to ./model.tar.gz


In [ ]:
import tarfile

with tarfile.open("./model.tar.gz", 'r:gz') as tar:
    tar.extractall(path="./")

In [ ]:
model.load_state_dict(torch.load("latest_model.pth"))

<All keys matched successfully>

In [ ]:
with open('latest_model.pth', 'rb') as f:
    model.load_state_dict(torch.load(f))

In [ ]:
model.eval()

Net(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(


In [ ]:
from sagemaker.pytorch import PyTorchModel

In [74]:
import sagemaker
from sagemaker import get_execution_role
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "project-3-cola"

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [75]:
model_path = f"s3://{bucket}/{prefix}/cola-distilbert-model-2023-09-30-15-01-17-574/output/model.tar.gz"

model = PyTorchModel(
    entry_point="inference.py",
    source_dir="../src",
    role=role,
    model_data=model_path,
    framework_version="1.10",
    py_version="py38",
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [76]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.BytesDeserializer(),
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
----------------!

In [87]:
import random
num = random.randint(0, len(valid_data))

print(num)
print(predictor.predict(valid_data[0]).decode(), valid_data[0]['label'])
print(predictor.predict(valid_data[num]).decode(), valid_data[num]['label'])

752
[[-3.7780814170837402, 4.0952277183532715]] 1
[[-3.5095458030700684, 3.72538423538208]] 1


In [88]:
valid_data[num]

{'idx': 752, 'label': 1, 'sentence': 'Our friends like us.'}

In [89]:
predictor.delete_endpoint()